<a href="https://colab.research.google.com/github/Fredrick-kyalo/Text_classification/blob/main/sentiment_prediction_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
#Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

print(tf.__version__)

2.12.0


In [110]:
%ls

Relations1.csv  sample_data/


In [111]:
dataset = pd.read_csv("Relations1.csv")
dataset.head()

df = dataset[['Tweet','Relations']]

print(df.shape)
df.head(5)

(11892, 2)


,Tweet,Relations
0,@QuinOtieno1 #SquadGoals 👌🏾 - Congratulations ...,friendship
1,Cheers to the ones who've been there through t...,friendship
2,"Finding joy in the little things, like playing...",friendship
3,Who's next?\r\n\r\n$Arch #BuildingBridges #Squ...,friendship
4,"I regret to inform you, but my team is better ...",friendship


In [112]:
df['Tweet'] = df['Tweet'].str.replace('[^\w\s]', '') # removes punctuation
df['Tweet'] = df['Tweet'].str.replace('\d+', '') # removes numbers
df['Tweet'] = df['Tweet'].str.replace('@', '') # removes leading whitespace '@'
df['Tweet'] = df['Tweet'].str.replace('#', '') # removes trailing '#'
df['Tweet'] = df['Tweet'].str.replace('\s+', '') # removes extra whitespace
df['Tweet'] = df['Tweet'].str.lower() # lowercase
df.head()

<ipython-input-112-4744b5a7a7c4>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet'] = df['Tweet'].str.replace('[^\w\s]', '') # removes punctuation
<ipython-input-112-4744b5a7a7c4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tweet'] = df['Tweet'].str.replace('[^\w\s]', '') # removes punctuation
<ipython-input-112-4744b5a7a7c4>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet'] = df['Tweet'].str.replace('\d+', '') # removes numbers
<ipython-input-112-4744b5a7a7c4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,Tweet,Relations
0,quinotienosquadgoalscongratulationstoyouall,friendship
1,cheerstotheoneswhovebeentherethroughthickandth...,friendship
2,findingjoyinthelittlethingslikeplayingludogian...,friendship
3,whosnextarchbuildingbridgessquadgoalshttpstcoo...,friendship
4,iregrettoinformyoubutmyteamisbetterthanyourstr...,friendship


In [113]:
train_texts =df['Tweet']
train_labels = df['Relations']

test_texts = df['Tweet']
test_labels =df['Relations']


df

,Tweet,Relations
0,quinotienosquadgoalscongratulationstoyouall,friendship
1,cheerstotheoneswhovebeentherethroughthickandth...,friendship
2,findingjoyinthelittlethingslikeplayingludogian...,friendship
3,whosnextarchbuildingbridgessquadgoalshttpstcoo...,friendship
4,iregrettoinformyoubutmyteamisbetterthanyourstr...,friendship
...,...,...
11887,rtfamilygoalspodareyouprayingforyourlostfriend...,family
11888,rtfamilygoalspodareyouprayingforyourlostfriend...,family
11889,theyarethestoryofwhoweareandourcollectionofthe...,family
11890,thealreadylegendarysammyprattistodayfamilyisev...,family


In [114]:
def enc(label):
    return 1 if label == 'real' else 0

# Apply the encoding function to create a new 'y' column in review_df
df['y'] = df['Relations'].apply(enc)



<ipython-input-114-b11f2d36deed>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['y'] = df['Relations'].apply(enc)


In [115]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(df['Tweet'], df['y'], test_size=0.2, random_state=42)


In [116]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
train_seq = tokenizer.texts_to_sequences(train_texts)
test_seq = tokenizer.texts_to_sequences(test_texts)

In [117]:
max_seq_length = max(max(map(len, train_seq)),  max(map(len, train_seq)))
print(max_seq_length)

17


In [118]:
train_data = pad_sequences(train_seq, maxlen = max_seq_length)
test_data = pad_sequences(test_seq, maxlen = max_seq_length)

sentiment_label = df.Relations.factorize()
sentiment_label

(array([0, 0, 0, ..., 2, 2, 2]),
 Index(['friendship', 'love', 'family'], dtype='object'))

In [119]:
print('GPU available: ', tf.config.list_physical_devices("GPU"))

GPU available:  []


In [120]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D

# Create the Sequential model
model = Sequential()

# Add an Embedding layer
model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_seq_length))

# Add a SpatialDropout1D layer
model.add(SpatialDropout1D(0.4))  # Experiment with dropout rate

# Add an LSTM layer
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))  # Experiment with dropout rates

# Add a Dense layer for output
model.add(Dense(1, activation='sigmoid'))

# Compile the model with different optimizer and learning rate
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)  # Experiment with learning rate
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Print the model summary
print(model.summary())


#print(model.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 17, 100)           979200    
                                                                 
 spatial_dropout1d_7 (Spatia  (None, 17, 100)          0         
 lDropout1D)                                                     
                                                                 
 lstm_8 (LSTM)               (None, 128)               117248    
                                                                 
 dense_8 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,096,577
Trainable params: 1,096,577
Non-trainable params: 0
_________________________________________________________________
None


In [125]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#history = model.fit(train_data, y=train_labels, validation_data=(test_data, test_labels), epochs=10, batch_size=64)

history = model.fit(train_data,sentiment_label[0],validation_split=0.1, epochs=10, batch_size=64)

Epoch 1/10
134/134 [==============================] - 32s 49ms/step - loss: 0.1166 - accuracy: 0.9565 - val_loss: 0.2781 - val_accuracy: 0.9816
Epoch 2/10
134/134 [==============================] - 7s 53ms/step - loss: 0.1063 - accuracy: 0.9579 - val_loss: 0.2140 - val_accuracy: 0.9811
Epoch 3/10
134/134 [==============================] - 6s 46ms/step - loss: 0.0996 - accuracy: 0.9585 - val_loss: 0.2069 - val_accuracy: 0.9832
Epoch 4/10
134/134 [==============================] - 7s 52ms/step - loss: 0.0942 - accuracy: 0.9583 - val_loss: 0.1964 - val_accuracy: 0.9837
Epoch 5/10
134/134 [==============================] - 6s 46ms/step - loss: 0.0896 - accuracy: 0.9585 - val_loss: 0.1823 - val_accuracy: 0.9848
Epoch 6/10
134/134 [==============================] - 7s 53ms/step - loss: 0.0870 - accuracy: 0.9584 - val_loss: 0.1686 - val_accuracy: 0.9842
Epoch 7/10
134/134 [==============================] - 7s 52ms/step - loss: 0.0851 - accuracy: 0.9581 - val_loss: 0.1578 - val_accuracy: 0.984

In [122]:
loss, acc = model.evaluate(test_data, test_labels)
print(loss)
print(acc)

75/75 [==============================] - 1s 5ms/step - loss: 1.7707 - accuracy: 0.0404
1.770690679550171
0.040353089570999146


In [132]:
def predict_sentiment(text):
    # Tokenize and pad the input text using the same tokenizer and max_seq_length
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw, maxlen=max_seq_length)  # Assuming you defined max_seq_length

    # Predict the sentiment label
    prediction = int(model.predict(tw).round().item())

    # Map the predicted label to its corresponding sentiment category
    sentiment_mapping = {0: 'friendship', 1: 'love', 2: 'family'}
    predicted_sentiment = sentiment_mapping[prediction]

    print("Predicted sentiment category: ", predicted_sentiment)

# Get user input for the test sentence
test_sentence = input("Enter a test sentence: ")
predict_sentiment(test_sentence)


Enter a test sentence: Have you heard that father is sick
1/1 [==============================] - 0s 19ms/step
Predicted sentiment category:  friendship
